### Generators
A generator is a function that produces a sequence of results instead of a single value.  
Instead of returning a value, you generate a series of values (using the yield statement)  
Typically, it is hooked to a for-loop.  
http://www.dabeaz.com/coroutines/Coroutines.pdf  


In [8]:
def countdown(n):
    print("Counting down from", n)
    while n > 0:
        yield n
        n -= 1

for i in countdown(5):
    print(i, end="   ")


Counting down from 5
5   4   3   2   1   

In [16]:
x = countdown(10)
print("x =", x, end="\n\n")

print("Calling first next(x)")
print("next(x) =", next(x))
print("Calling first next(x)")
print("next(x) =", next(x))

x = <generator object countdown at 0x000000000584D990>

Calling first next(x)
Counting down from 10
next(x) = 10
Calling first next(x)
next(x) = 9


In [23]:
# A Python version of Unix 'tail -f'

import time
def follow(thefile):
    thefile.seek(0,2) # Go to the end of the file
    while True:
        line = thefile.readline()
        if not line:
            time.sleep(0.1) # Sleep briefly
            continue
    yield line
#  Floowing code will excersize this generator (but it is an infinite generator, so will have to kill)
logfile = open("tmpxxx/out.log")
for line in follow(logfile):
    print(line)

FileNotFoundError: [Errno 2] No such file or directory: 'tmpxxx/out.log'

#### Generators as Pipelines
One of the most powerful applications of generators is setting up processing pipelines (Similar to shell pipes in Unix).  
You can stack a series of generator functions together into a pipe and pull items through it with a for-loop

In [22]:
def grep(pattern, lines):
    for line in lines:
        if pattern in line:
            yield line

# Set up a processing pipe : tail -f | grep python
logfile = open("tmpxxx/out.log")
loglines = follow(logfile)
pylines = grep("python", loglines)
# Pull results out of the processing pipeline
for line in pylines:
    print(line)


FileNotFoundError: [Errno 2] No such file or directory: 'tmpxxx/out.log'

#### Begining of coreutines
yield can be used as an expression. For example, on the right side of an assignment

In [24]:
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)

If you use yield as an expression, you get a coroutine.  
These do more than just generate values.  
Instead, functions can consume values sent to it.

In [27]:
g = grep("python")
# Prime it. next() advances the coroutine to the first yield expression
next(g)
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")


Looking for python
python generators rock!


In [33]:
#  Remembering to call next() is easy to forget.
#  Solved by wrapping coroutines with a decorator.

def coroutine(func):
    def start(*args,**kwargs):
        cr = func(*args,**kwargs)
        next(cr)
        return cr
    return start

@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    try:
        while True:
            line = (yield)
            if pattern in line:
                print(line)
    except GeneratorExit: # To catch close
        print("Going away. Goodbye")

g = grep("python")
# next(g) not needed as coroutine decorator takes care of it.
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")
g.close()


Looking for python
Going away. Goodbye
python generators rock!
Going away. Goodbye


Generators produce data for iteration.  
Coroutines are consumers of data.  